In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import albumentations # Library to do augmentation on images
import time
import os
import PIL
from PIL import Image

# Graphs
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Sk Learn
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset

#SciKit Learn
from sklearn.model_selection import train_test_split
from sklearn import metrics, model_selection
import torch
from torchvision.datasets.utils import download_url

# Metrics
from sklearn.metrics import mean_squared_error

In [ ]:
DATASET_URL = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
DATA_FILENAME = "BostonHousing.csv"
download_url(DATASET_URL, '.')
# dataframe = pd.read_csv("../input/bostonhoustingmlnd/housing.csv")
df = pd.read_csv(DATA_FILENAME)
df.head()

In [ ]:
# df = pd.read_csv("../input/bostonhoustingmlnd/housing.csv")
# df.head()

In [ ]:
# inputs = df[['RM','LSTAT','PTRATIO']].values
# print(inputs)
# targets = df['MEDV'].values


inputs = df.drop('medv', axis=1).values
targets = df[['medv']].values

inputs = torch.tensor(inputs,dtype=torch.float32)
targets = torch.tensor(targets,dtype=torch.float32)

print(inputs.shape)
print(targets.shape)

In [ ]:
# train_ds = TensorDataset(inputs[:10], targets[:10])
from torch.utils.data import DataLoader, TensorDataset, random_split

dataset = TensorDataset(inputs,targets)
train_ds, val_ds = random_split(dataset, [406, 100])
batch_size = 64
val_loader = DataLoader(val_ds, batch_size*2)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
input_size = 13
output_size= 1

# for x,y in train_b1:
#     print(x)
#     print(y)

In [ ]:
import torch.nn as nn

class StoneModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, xb):
        out = self.linear(xb)
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        out = self(inputs)                 # Generate predictions
        loss = F.mse_loss(out, targets)    # Calculate loss
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch 
        out = self(inputs)                 # Generate predictions
        loss = F.mse_loss(out, targets)    # Calculate loss
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}".format(epoch, result['val_loss']))
    

model = StoneModel()

In [ ]:
import torch.nn.functional as F
loss_fn = F.mse_loss
list(model.parameters())

In [ ]:
lrate=5e-7
opt = torch.optim.SGD(model.parameters(), lr=lrate)

In [ ]:
def evaluate(model,)

In [ ]:
def fits(model,opt,epoch):
    
    for i in range (epoch):
        for x in train_loader:
            loss = model.training_step(x)
            loss.backward()
            opt.step()
            opt.zero_grad()
            print(loss.item())            

In [ ]:
fits(model,opt,4000)

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
result = evaluate(model, val_loader)
result

In [ ]:
history = fit(10, lrate, model, train_loader, val_loader)

In [ ]:
losses = [r['val_loss'] for r in [result] + history]
plt.plot(losses, '-x')
plt.xlabel('epoch')
plt.ylabel('val_loss')
plt.title('val_loss vs. epochs');